In [2]:
pip install pandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

Reading the DataSet 

In [6]:
data_df = pd.read_csv('ASIANPAINT.csv')
data_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-02-02,989.450012,995.000000,981.950012,989.000000,958.942017,1365514
1,2017-02-03,990.000000,993.900024,981.049988,985.900024,955.936096,465754
2,2017-02-06,990.099976,1004.950012,985.500000,992.500000,962.335571,744278
3,2017-02-07,992.000000,1003.950012,990.200012,999.299988,968.928833,577723
4,2017-02-08,999.299988,1009.900024,981.000000,997.700012,967.377563,1192386
...,...,...,...,...,...,...,...
1230,2022-01-25,3100.000000,3156.949951,3010.750000,3147.100098,3147.100098,2699779
1231,2022-01-27,3140.100098,3143.000000,3071.250000,3116.949951,3116.949951,1834684
1232,2022-01-28,3130.000000,3170.000000,3103.000000,3110.850098,3110.850098,892060
1233,2022-01-31,3140.000000,3193.449951,3136.399902,3152.250000,3152.250000,944530


In [7]:
data_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-02-02,989.450012,995.000000,981.950012,989.000000,958.942017,1365514
1,2017-02-03,990.000000,993.900024,981.049988,985.900024,955.936096,465754
2,2017-02-06,990.099976,1004.950012,985.500000,992.500000,962.335571,744278
3,2017-02-07,992.000000,1003.950012,990.200012,999.299988,968.928833,577723
4,2017-02-08,999.299988,1009.900024,981.000000,997.700012,967.377563,1192386


In [8]:
data_df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1.235000e+03
mean,1784.580607,1804.636760,1763.160690,1783.614210,1762.049029,1.448089e+06
std,699.204529,706.664895,690.499606,698.234663,707.391939,1.112168e+06
min,961.000000,973.400024,953.849976,960.799988,931.598938,7.026400e+04
25%,1213.950012,1231.325012,1196.200012,1215.575012,1183.003357,7.508415e+05
50%,1514.000000,1531.449951,1495.150024,1519.300049,1491.635620,1.137313e+06
75%,2166.675049,2199.349976,2153.074951,2168.000000,2154.776001,1.766695e+06
max,3580.000000,3590.000000,3534.000000,3576.300049,3576.300049,1.154592e+07


In [9]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1235 non-null   object 
 1   Open       1235 non-null   float64
 2   High       1235 non-null   float64
 3   Low        1235 non-null   float64
 4   Close      1235 non-null   float64
 5   Adj Close  1235 non-null   float64
 6   Volume     1235 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 67.7+ KB


In [11]:
data_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [12]:
data_df.count()

Date         1235
Open         1235
High         1235
Low          1235
Close        1235
Adj Close    1235
Volume       1235
dtype: int64

In [16]:
data_df.isnull()

,Date,Open,High,Low,Close,Adj Close,Volume
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
1230,False,False,False,False,False,False,False
1231,False,False,False,False,False,False,False
1232,False,False,False,False,False,False,False
1233,False,False,False,False,False,False,False
